# Introduction to Programming for Artificial Intelligence and Data                                 Science Assessment 1
## Customer Data Pre-processing
### By
### Joshua Ejiofor Abonyi
### Student ID: 202333142

## Introduction
An extensive collection of data named "acw_user_data" has been provided in CSV file format. It contains customer records
which is to be processed and analysed for filtering and metrics derivation.

#### First, the python libraries need to be imported:

In [1]:
# Importing the necessary libraries

import csv
import json
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

## Data Processing

### 1. Reading in the provided ACW Data using the CSV library

In [2]:
# Using try and exception to handle errors

try:
    # Initialising an empty list to store customer data
    customer_data = []

    # Opening the CSV file for reading
    with open('acw_user_data.csv', mode='r') as csv_file:
        # Creating a CSV reader object with the specified delimiter
        csv_reader = csv.reader(csv_file, delimiter=',')

        # Looping through each row in the CSV file and appending it to the customer_data list
        for dt in csv_reader:
            customer_data.append(dt)

# Catching and handling the FileNotFoundError
except FileNotFoundError as file_err:
    # Printing an error message if the file is not found
    print(file_err, 'Uh oh! Forgot to place the file where the Jupyter Notebook is saved.')

# If no error occurs, this block will execute
else:
    # Printing the first few rows of the customer_data list to verify successful data loading
    # This approach avoids printing the entire dataset, which can be very large
    print("First few rows of data:", customer_data[:5])

#### - It is necessary to know the number of columns in the dataset.
This can be done by enumerating the list containing it. 

In [3]:
# Numbering the list of column names
for line_num, dt in enumerate(customer_data[0]):
    print(line_num, dt)

### 2. Converting the CSV flat file to a nested structure

In [4]:
# Initialising an empty list
structured_data = []

# Iterating over each row in the dataset, skipping the header row
for row in customer_data[1:]:
    # Creating a dictionary for each customer with nested structures for Vehicle, Credit Card, and Address
    person = {
        'FirstName': row[11],
        'LastName': row[13],
        'Age': int(row[3]),
        'DistanceCommuted': float(row[4]),
        'Employer': row[5],
        'Dependants': int(row[10]) if row[10].isdigit() else 0,  # Defaulting to 0 if not a digit
        'MaritalStatus': row[14],
        'YearlyPension': float(row[15]),
        'Retired': row[16],
        'YearlySalary': float(row[17]),
        'Sex': row[18],
        'Vehicle': {
            'Make': row[19],
            'Model': row[20],
            'Year': int(row[21]),
            'Type': row[22]
        },
        'CreditCard': {
            'StartDate': row[6],
            'ExpiryDate': row[7],
            'Number': row[8],
            'CVV': row[9],
            'IBAN': row[12]
        },
        'Address': {
            'Street': row[0],
            'City': row[1],
            'Postcode': row[2]
        }
    }
    structured_data.append(person)  # Adding the structured person data to the list

### Print the output


In [5]:
structured_data

### 3. Writing all records to a `processed.json` file in the JSON data format
The programme went ahead to check that the written file is uncompromised by first reading it and setting the read data `True` against the `acw_data` that was dumped in it initially.

In [6]:
with open('processed.json', mode='w') as json_file:    #create, open, write
    json.dump(structured_data, json_file, indent=4)
    
with open('processed.json', mode='r') as json_file:    #open, read, assign
    structured_data_2 = json.load(json_file)
    
if len(structured_data) == len(structured_data_2):    #checking if its intact
    print("True")

### 4. Creating two additional file outputs, `retired.json` and `employed.json`

In [7]:
# Filtering out retired customers
retired = [person for person in structured_data if person['Retired'] == 'True']
# Filtering out employed customers
employed = [person for person in structured_data if person['Employer'] != '']

# Writing the filtered data to respective JSON files
with open('retired.json', 'w') as file:
    json.dump(retired, file, indent=4)

with open('employed.json', 'w') as file:
    json.dump(employed, file, indent=4)

### 5. Credit Card Validation

In [8]:
#stackoverflow, 2023

# Writing a function to check validity of credit card based on the start and expiry dates
def is_credit_card_valid(person):
    try:
        # Adjusting the format to match date format, e.g., 'MM/YY'
        start_date = datetime.strptime(person['CreditCard']['StartDate'], '%m/%y')
        expiry_date = datetime.strptime(person['CreditCard']['ExpiryDate'], '%m/%y')

        # Adjusting the years if they are provided in two digits
        # Assuming the century is 2000 if the year is less than 70 (e.g., '69' becomes 2069, '18' becomes 2018)
        start_date = start_date.replace(year=start_date.year + 100) if start_date.year < 1970 else start_date
        expiry_date = expiry_date.replace(year=expiry_date.year + 100) if expiry_date.year < 1970 else expiry_date

        return (expiry_date - start_date).days <= 3650  # Check if the date difference is less than or equal to 10 years
    except ValueError:
        # Handling cases where the date format might be incorrect or missing
        return False  

# Filtering customers with invalid credit cards
invalid_credit_cards = [person for person in structured_data if not is_credit_card_valid(person)]

# Writing these customers to a JSON file
with open('remove_ccard.json', 'w') as file:
    json.dump(invalid_credit_cards, file, indent=4)

### 6. Calculating some additional metrics

In [9]:
# Python Software Foundation 2023

import copy

# Creating a deep copy of structured_data
structured_data_with_metric = copy.deepcopy(structured_data)

# Calculating the 'Salary-Commute' metric for each person in the copied list
for person in structured_data_with_metric:
    commute_distance = person['DistanceCommuted']
    salary = person['YearlySalary']
    # Calculating salary per km of commute
    person['Salary-Commute'] = salary if commute_distance <= 1 else salary / commute_distance

# Sorting the data in the copied list based on the 'Salary-Commute' metric
structured_data_with_metric.sort(key=lambda x: x['Salary-Commute'])

# Writing the sorted data to a JSON file
with open('commute.json', 'w') as file:
    json.dump(structured_data_with_metric, file, indent=4)

### Let's have a peak at the sorted data

In [10]:
structured_data_with_metric

## Data Visualisation
### The programme used pandas and seaborn for the visualisation below:

In [11]:
import pandas as pd
import seaborn as sns

In [12]:
# Reading the CSV file into a Pandas DataFrame
acw_data_frame = pd.read_csv('acw_user_data.csv')  #reading in the CSV file with pandas
acw_data_frame

In [13]:
print(type(acw_data_frame))  #confirming the datatype

In [14]:
acw_data_frame.isnull().any()  #checking for any null values

## 1. Obtaining the data series for salary and age

In [15]:
# Data Series for Salary and Age
salary_series = acw_data_frame['Yearly Salary (GBP)']
age_series = acw_data_frame['Age (Years)']

# Printing the Data Series for Salary and Age
print("Salary Series:")
print(salary_series)
print("\nAge Series:")
print(age_series)

#### Calculating their respective mean and median 

In [16]:
# Calculating Mean Salary and Median Age
mean_salary = salary_series.mean()
median_age = age_series.median()

print(f"Mean Salary of Customers = £{mean_salary:.2f}")
print(f"Median Age of Customers = {round(median_age)} years")

## 2. Univariate Plots
a) Age Distribution with Bin Width of 5

In [17]:
#QiMacros, 2020
# Determining the number of bins with a bin width of 5
max_age = acw_data_frame['Age (Years)'].max()
min_age = acw_data_frame['Age (Years)'].min()
num_bins = (max_age - min_age) // 5 + 1
print(f"Number of bins for Age with a bin width of 5: {num_bins}")

In [18]:
# Plotting Age Distribution
plt.figure(figsize=(10, 6))
univar_age = sns.histplot(acw_data_frame['Age (Years)'], bins=num_bins)
plt.title('Age Distribution')
plt.xlabel('Age (Years)')
plt.ylabel('Count')
plt.show()

b) Dependents Count Plot (Fixing Data Errors)

In [19]:
# Converting to numeric, filling non-numeric with NaN, then replace NaN with 0
acw_data_frame['Dependants'] = pd.to_numeric(acw_data_frame['Dependants'], errors='coerce').fillna(0)

# Printing the corrected 'Dependants' series
print("Corrected Dependents Series:")
print(acw_data_frame['Dependants'].head())

# Plotting Dependents Count
plt.figure(figsize=(10, 6))
univar_dependants = sns.countplot(x='Dependants', data=acw_data_frame)
plt.title('Dependents Count')
plt.xlabel('Number of Dependents')
plt.ylabel('Count')
plt.show()

c) Age Distribution on Marital Status

In [20]:
#stackoverflow,2023

# Plotting Age Distribution Conditioned on Marital Status
plt.figure(figsize=(10, 6))
age_married_graph = sns.histplot(acw_data_frame, x='Age (Years)', hue='Marital Status', multiple='stack')
plt.title('Age Distribution by Marital Status')
plt.xlabel('Age (Years)')
plt.ylabel('Count')
plt.show()

#### 3. Performing multivariate plots for:
a. `Commuted distance` against `Salary`

b. `Age` against `Salary`

c. `Age` against `Salary` conditioned by `Dependants`

In [21]:
#a. `Commuted distance` against `Salary`

# Scatter plot for Commuted Distance vs. Salary
plt.figure(figsize=(10, 6))
multivar_salary_commute = sns.scatterplot(x='Distance Commuted to Work (Km)', y='Yearly Salary (GBP)', data=acw_data_frame)
plt.title('Commuted Distance vs. Salary')
plt.xlabel('Distance Commuted to Work (Km)')
plt.ylabel('Yearly Salary (GBP)')
plt.show()

In [22]:
#b. `Age` against `Salary`

# Scatter plot for Age vs. Salary
plt.figure(figsize=(10, 6))
multivar_salary_age = sns.scatterplot(x='Age (Years)', y='Yearly Salary (GBP)', data=acw_data_frame)
plt.title('Age vs. Salary')
plt.xlabel('Age (Years)')
plt.ylabel('Yearly Salary (GBP)')
plt.show()

In [23]:
#stackoverflow, 2023 (colours)
#`Age` against `Salary` conditioned by `Dependants`

# Scatter plot for Age vs. Salary conditioned by the number of Dependents
plt.figure(figsize=(10, 6))
multivar_salary_age_dependants = sns.scatterplot(x='Age (Years)', y='Yearly Salary (GBP)', hue='Dependants', palette=["C3", "g", "C1", "k", "b"], data=acw_data_frame)
plt.title('Age vs. Salary (Conditioned by Dependents)')
plt.xlabel('Age (Years)')
plt.ylabel('Yearly Salary (GBP)')
plt.legend(title='Dependants', loc='upper left')
plt.show()

#### 4. Saving Plots
Below is a dictionary created to feed the `savefig()` function in order to save the graphs produced during the data visualisation

In [24]:
# Dictionary to store the figure objects
acw_figures = {}  

# Assuming 'univar_age', 'univar_dependants', etc. are Axes objects returned from seaborn plotting functions
acw_figures["./univariate_age.png"] = univar_age.figure
acw_figures["./univariate_dependants.png"] = univar_dependants.figure
acw_figures["./univariate_age_marital_status.png"] = age_married_graph.figure
acw_figures["./multivariate_salary_commute.png"] = multivar_salary_commute.figure
acw_figures["./multivariate_salary_age.png"] = multivar_salary_age.figure
acw_figures["./multivariate_salary_age_dependants.png"] = multivar_salary_age_dependants.figure

# Looping through the dictionary and saving each plot
for fig_path, fig in acw_figures.items():
    fig.savefig(fig_path)

### REFERENCES

##### 1. QiMacros, 2020. *"How to Determine Histogram Bin Width and Bin Intervals"*. Available Online: https://www.qimacros.com/histogram-excel/how-to-determine-histogram-bin-interval/ [Accessed 05/11/2022].

##### 2. Stackoverflow, 2023. *"Seaborn - Change Color According to Hue Name"*. Available online: https://stackoverflow.com/questions/46173419/seaborn-change-color-according-to-hue-name/[Accessed 05/11/2022].

##### 3. Stackoverflow, 2023. *"How do I sort a list of dictionaries by a value of the dictionary?"*. Available online: https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-a-value-of-the-dictionary/[Accessed 02/11/2022].

##### 4. Python Software Foundation. "copy — Shallow and deep copy operations." Python Standard Library, 2023. Accessed on [date you accessed the information]. URL: https://docs.python.org/3/library/copy.html.

##### 5. Stackoverflow, 2023. *"How to change a datetime format in python?"*. Available online: https://stackoverflow.com/questions/64137532/how-to-change-a-datetime-format-in-python/[Accessed 01/11/2022].

##### 6. Stackoverflow, 2023. *"How can I reorder a list? [closed]"*. Available online: https://stackoverflow.com/questions/2177590/how-can-i-reorder-a-list/[Accessed 24/10/2022].